In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from IPython import display
#from tensorflow.data import Dataset as tfds
from tensorflow.keras.utils import plot_model
import IPython

tf.compat.v1.reset_default_graph()
#import tensorflow.datasets as tfds
%matplotlib inline

In [2]:
#隨機產產生30張圖片的function
def generate_images():
    #隨機產生noise
    glabels = np.zeros(30)
    for i in range(30):
        if i%10==0:
            glabels[i]=0
        if i%10==1:
            glabels[i]=1
        if i%10==2:
            glabels[i]=2

    #產生圖片
    noise = tf.random.normal([30, latent_dim])
    generated_images = (conditional_gen([glabels,noise]))

    #將generated_images值縮為0-1
    immax = np.max(generated_images)
    immin = np.min(generated_images)
    if immin < 0:
        generated_images += abs(immin)
    generated_images /= (immax - immin)
    
    return generated_images

In [3]:
#印出10張圖片的function
def display_3class(array1, savepath):
    n = 10

    plt.figure(figsize=(20, 6))
    for i in range(n):
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(array1[i])
        plt.title('cat')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(array1[i+10])
        plt.title('dog')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        ax = plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(array1[i+20])
        plt.title('panda')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
    plt.gcf()
    plt.savefig(savepath)    
    plt.show()

In [5]:
conditional_gen = tf.keras.models.load_model('generator.h5')
conditional_discriminator = tf.keras.models.load_model('discriminator.h5')

In [6]:
#用於儲存d和g的loss
d1_loss = 0
d2_loss = 0
d_loss = []
g_loss = []

#define train_step
# Notice the use of `tf.function`. This annotation causes the function to be "compiled".
@tf.function(tf.config.run_functions_eagerly(True))
def train_step(images,target):
    # noise vector sampled from normal distribution
    noise = tf.random.normal([target.shape[0], latent_dim])
    # Train Discriminator with real labels
    with tf.GradientTape() as disc_tape1:
        generated_images = conditional_gen([target,noise], training=True)

        real_output = conditional_discriminator([images,target], training=True)
        real_targets = tf.ones_like(real_output)
        disc_loss1 = discriminator_loss(real_targets, real_output)
        d1_loss = disc_loss1.numpy()
        
    # gradient calculation for discriminator for real labels    
    gradients_of_disc1 = disc_tape1.gradient(disc_loss1, conditional_discriminator.trainable_variables)
    
    # parameters optimization for discriminator for real labels   
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc1,\
    conditional_discriminator.trainable_variables))
    
    # Train Discriminator with fake labels
    with tf.GradientTape() as disc_tape2:
        fake_output = conditional_discriminator([generated_images,target], training=True)
        fake_targets = tf.zeros_like(fake_output)
        disc_loss2 = discriminator_loss(fake_targets, fake_output)
        d2_loss = disc_loss2.numpy()
        
    #
    d_loss.append(d1_loss + d2_loss)
    
    # gradient calculation for discriminator for fake labels 
    gradients_of_disc2 = disc_tape2.gradient(disc_loss2, conditional_discriminator.trainable_variables)
    
    
    # parameters optimization for discriminator for fake labels        
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc2,\
    conditional_discriminator.trainable_variables))
    
    
    # Train Generator with real labels
    with tf.GradientTape() as gen_tape:
        generated_images = conditional_gen([target,noise], training=True)
        fake_output = conditional_discriminator([generated_images,target], training=True)
        real_targets = tf.ones_like(fake_output)
        gen_loss = generator_loss(real_targets, fake_output)
        g_loss.append(gen_loss.numpy())

    # gradient calculation for generator for real labels     
    gradients_of_gen = gen_tape.gradient(gen_loss, conditional_gen.trainable_variables)
    
    
    # parameters optimization for generator for real labels
    generator_optimizer.apply_gradients(zip(gradients_of_gen,\
    conditional_gen.trainable_variables))  
    
    
    #print("g_loss: {gen_loss}, d_loss: {disc_total_loss}")


In [7]:
#train
import time
def train(dataset, epochs):
    for epoch in range(epochs):
        iteration = 0
        start = time.time()
        for image_batch in dataset:
            img = tf.cast(image_batch[0], tf.float32)
            #imgs = normalization(img)
            imgs = img
            train_step(imgs, image_batch[1])
            print ('epoch {}, {}/{}, g_loss: {} d_loss: {}'.format(epoch + 1, iteration+1, len(dataset), g_loss[-1], d_loss[-1]))
            iteration+=1
        
        conditional_gen.save('generator.h5')
        conditional_discriminator.save('discriminator.h5')
        print('epoch{}: {}sec\n'.format(epoch+1, round(time.time()-start)))
        
        if epoch%10==0 or epoch%epochs==0:
            epoch_imgs = generate_images()
            savepath =  "./animal_generate/epoch_"+str(epoch)+"_animals_generate.jpg" 
            display_3class(epoch_imgs, savepath)
              
epoch = 200
train(dataset, epoch)


NameError: name 'dataset' is not defined